# SISTEMAS DE RECOMENDACIÓN

Los sistemas de recomendación son modelos cuyo objetivo es proporcionar, como su nombre indica, una recomendación de un producto basada en la experiencia el consumidor o considerando en conjunto la experiencia del consumidor junto con otros.

Tradicionalmente, los sistemas de recomendación se han basado en la creación de rankings, de modo que se recomienda aquello que más ha votado una comunidad de usuarios.

Gracias a la evolución de los algoritmos de inteligencia artificial, se plantea la posiblidad de aplicar métodos más complejos para la obtención de recomendaciones más precisas y personalizadas.

Así surgen, entre otros, los llamados sistemas colaborativos, que intentan obtener un perfilado o tratar de asignar recomendaciones basadas en perfiles similares.

Entre estos últimos podemos destacar los algoritmos basados en distancias y las descomposiciones matriciales. 

En el ejemplo que se va a desarrollar se va a mostrar un ejemplo tanto de de sistema basado en un ranking genérico y un ejemplo más detallado con un sistema de distancias.

# ANÁLISIS DEL DATASET

Vamos a utilizar un dataset de la plataforma Kaggle que consiste en un conjunto de películas y usuarios los cuales han puntuado entre 0 y 5 las películas que han visto.

Los datos se componen de dos archivos:
- ratings_small.csv con la relación usuario - identificador de película - puntuación
- movies_medatada.csv donde hay información asociada de la película como su nombre y género.

In [2]:
### Carga de librerías genéricas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

plt.style.use("ggplot")
warnings.filterwarnings("ignore")

In [3]:
##Direcciones de los archivos

camino="D:\\J\\Big data\\DATAHACK\\PYTHON\\006-_Caso_segmentacion\\Recomendador peliculas\\"
archivo="ratings_small.csv"
archivo_pelis="movies_metadata.csv"

In [4]:
#Lectura del fichero con los ratings
df=pd.read_csv(camino+archivo)

In [5]:
# Información asociada a las películas
df_info_pelis=pd.read_csv(camino+archivo_pelis)

## ANÁLISIS DE LOS DATAFRAMES

Procedemos a analizar los dos daframes y su estructura, comenzando por el de los rankings

In [6]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


El usuaro se identifica con un id, así como la película- El rating es un número, veremos entre qué se mueve- El timestamp no lo vamos a utilizar, con lo que podemos eliminarlo en primer lugar

In [7]:
df.drop(columns=["timestamp"],inplace=True)

In [8]:
df["rating"].value_counts().sort_index(ascending=False)

5.0    15095
4.5     7723
4.0    28750
3.5    10538
3.0    20064
2.5     4449
2.0     7271
1.5     1687
1.0     3326
0.5     1101
Name: rating, dtype: int64

En efecto, se ve que los ratings van desde 0.5 hasta 5. Veamos cuántos usuarios distintos hay, así como películas:

In [9]:
df["userId"].unique().shape

(671,)

Hay 671 usuarios diferentes

In [10]:
### ESCRIBE TU CÓDIGO AQUÍ PARA OBTENER EL NÚMERO DE PELÍCULAS DISTINTAS

df["movieId"].unique().shape

(9066,)

Hay 9066 películas diferentes. Veamos ahora el archivo de las películas:

In [11]:
df_info_pelis.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [12]:
df_info_pelis.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

De todas las columnas nos interesan la que tiene el género de la película, el id para poder cruzar, el título y la media de votos. Filtraomos el df por estos campos.

In [13]:
###TU CÓDIGO PARA QUEDARNOS SOLO CON LAS COLUMNAS:["genres","id","title","vote_average"]
### MODIFICA EL DF df_info_pelis 

df_info_pelis=df_info_pelis[["genres","id","title","vote_average"]]
df_info_pelis.head()

,genres,id,title,vote_average
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story,7.7
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,Jumanji,6.9
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,Grumpier Old Men,6.5
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,Waiting to Exhale,6.1
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Father of the Bride Part II,5.7


La columna "genres" tiene una estructura un tanto complicada que habría que limpiar

In [14]:
df_info_pelis["genres"][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

Vemos que una película se puede clasificar de diferentes maneras. Para explotarlo tenemos que ver el tipo de columna:

In [15]:
df_info_pelis["genres"].dtype

dtype('O')

Podemos detectar el género porque se ve que tienen la primera mayúscula. Así podemos filtrar la lista de modo sencillo porque python identifica este formato en textos con el método .istitle()

In [65]:
print("Hola".istitle())
print("hola".istitle())

True
False


La siguiente línea de código crea una nueva columna solo con los géneros:

In [66]:
df_info_pelis["generos"]=df_info_pelis["genres"].apply(lambda x: [y for y in x.split("'") if y.istitle()])

Sin embargo, sigue siendo poco manejable, pues el elemento de la columna es un poco "extraño":


In [73]:
##TU CÓDIGO AQUÍ VIENDO EL TIPO DE ELEMENTO DE ESTA COLUMNA.
## AQUÍ DTYPE NO VA A FUNCIONAR PORQUE TE VA A DEVOLVER ALGO GENÉRICCO TIPO OBJECT.
## SACA UN ELEMENTO DE LA COLUMNA Y UTILIZA type

type(df_info_pelis["generos"][0])

list

Tenemos listas que habría que explotar, pues parece que es compatible que una película sea de animación y de aventuras a la vez, pero ¿cuántos es el máximo número de géneros para una sola película?

In [74]:
np.max(df_info_pelis["generos"].apply(lambda x: len(x)))

8

En efecto, se ve que hay hasta 8 géneros distintos para una sola película. La siguiente línea de código nos va a indicar cuántos distintos hay:

In [39]:
set([y for x in df_info_pelis["generos"].values for y in x])

{'Action',
 'Adventure',
 'Animation',
 'Aniplex',
 'Carousel Productions',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Mardock Scramble Production Committee',
 'Music',
 'Mystery',
 'Odyssey Media',
 'Pulser Productions',
 'Rogue State',
 'Romance',
 'Science Fiction',
 'Sentai Filmworks',
 'Telescene Film Group Productions',
 'The Cartel',
 'Thriller',
 'Vision View Entertainment',
 'War',
 'Western'}

La idea ahora es crear una lista con los que entedamos que son principales y después quedarnos solo con el primero. Crearemos una categría nueva con "otros" para los que no sean de ningún tipo. Sin embargo, antes vamos a limpiar el dataframe eliminando aquellas películas que no tienen ningún género asociado. La línea siguiente es la que lo hace:

In [75]:
df_info_pelis=df_info_pelis[df_info_pelis["generos"].apply(lambda x: bool(x))]

Ahora creamos la columna "genero" con el seleccionado:

In [90]:
nivel_1=['Pulser Productions',"Action","Animation","Adventure","Comedy",'Crime','Documentary','Fantasy','Music','Horror','Mystery','Romance','Science Fiction','Thriller','War','Western']

df_info_pelis["genero"]=df_info_pelis["generos"].apply(lambda x: [y if y in nivel_1 else "Otros" for y in x ][0])

¿Cuántos hay en cada tipo?

In [91]:
###Escribe a tu código. Utiliza value_counts() en la columna:

df_info_pelis["genero"].value_counts()

Otros              13045
Comedy              8869
Action              4521
Documentary         3428
Horror              2634
Crime               1707
Thriller            1680
Adventure           1522
Romance             1220
Animation           1131
Fantasy              714
Science Fiction      655
Mystery              561
Music                491
Western              458
War                  381
Name: genero, dtype: int64

In [46]:
df_info_pelis["generos"]

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: generos, Length: 45466, dtype: object

In [34]:
for x in df_info_pelis["generos"].values:
    for y in x:
        print(y)

Animation
Comedy
Family
Adventure
Fantasy
Family
Romance
Comedy
Comedy
Drama
Romance
Comedy
Action
Crime
Drama
Thriller
Comedy
Romance
Action
Adventure
Drama
Family
Action
Adventure
Thriller
Adventure
Action
Thriller
Comedy
Drama
Romance
Comedy
Horror
Family
Animation
Adventure
History
Drama
Action
Adventure
Drama
Crime
Drama
Romance
Crime
Comedy
Crime
Comedy
Adventure
Action
Comedy
Crime
Comedy
Thriller
Crime
Drama
Thriller
Action
Adventure
Crime
Thriller
Drama
Fantasy
Science Fiction
Thriller
Drama
Romance
Drama
Comedy
Drama
Family
Drama
Romance
Fantasy
Science Fiction
Adventure
Drama
Crime
Drama
Crime
Science Fiction
Thriller
Mystery
Romance
Adventure
Fantasy
Drama
Comedy
Family
History
Drama
Romance
Drama
Adventure
History
Drama
Family
Comedy
Family
Romance
Comedy
Drama
Romance
Drama
Drama
War
Action
Crime
Drama
History
Drama
Romance
Action
Fantasy
Fantasy
Drama
Comedy
Thriller
Drama
Romance
Crime
Mystery
Thriller
Adventure
Animation
Drama
Family
Drama
Romance
Drama
Crime
Thriller


Drama
War
Crime
Drama
Comedy
Foreign
Western
Drama
Romance
War
Comedy
Romance
War
Drama
Drama
Drama
Comedy
Drama
Romance
Mystery
Documentary
Crime
Comedy
Crime
Drama
Romance
Documentary
Music
Documentary
Drama
Comedy
Adventure
Action
Science Fiction
Adventure
Family
Fantasy
Comedy
Romance
Adventure
Action
Comedy
Science Fiction
Horror
Action
Thriller
Science Fiction
Western
Comedy
Drama
Drama
Adventure
Drama
History
War
Science Fiction
Drama
Crime
Drama
Drama
War
Western
Adventure
Action
Science Fiction
Drama
Fantasy
Romance
Thriller
Mystery
Drama
Crime
Horror
Action
Thriller
Science Fiction
Fantasy
Horror
Comedy
Adventure
Drama
Romance
Action
Drama
History
Action
Crime
Drama
Thriller
Drama
Horror
Thriller
Music
Comedy
Action
Crime
Drama
Crime
Drama
War
Adventure
Animation
Comedy
Science Fiction
Family
War
Drama
History
Action
Romance
Drama
History
Music
Comedy
Drama
Romance
Drama
Crime
Drama
Comedy
Drama
Romance
Drama
History
Drama
Science Fiction
Thriller
Action
Drama
History
War
Adv

Adventure
Thriller
Adventure
Animation
Drama
Family
Music
Comedy
Drama
Romance
History
Drama
Drama
Family
Family
Fantasy
Adventure
Science Fiction
Comedy
Drama
Family
Fantasy
Romance
Western
Action
Adventure
Thriller
War
Action
Adventure
Thriller
War
Action
Adventure
Thriller
War
Action
Adventure
Comedy
Romance
Action
Adventure
Comedy
Romance
Comedy
Drama
Science Fiction
Comedy
Drama
Science Fiction
Drama
Drama
Drama
Drama
Comedy
Thriller
Crime
Mystery
Action
Adventure
Crime
Drama
Family
Mystery
Thriller
Comedy
Comedy
Drama
Romance
Drama
Comedy
Romance
Drama
Thriller
Drama
Adventure
Drama
Action
Romance
Family
Family
Adventure
Comedy
Comedy
Romance
Drama
Action
Crime
Drama
Drama
History
War
Horror
Mystery
Science Fiction
Action
Adventure
Family
Fantasy
Adventure
Fantasy
Drama
Action
Science Fiction
Comedy
Drama
Drama
Romance
Drama
Drama
Comedy
Drama
Comedy
Drama
War
Drama
History
Thriller
Comedy
Drama
Romance
Drama
Thriller
Crime
Drama
Drama
Action
Western
Romance
Music
Drama
Drama
Com

Fantasy
Drama
Comedy
Drama
Family
Animation
Drama
Family
Science Fiction
Mystery
Action
Science Fiction
War
Drama
Music
Comedy
Action
Comedy
Comedy
Romance
Drama
Comedy
Fantasy
Music
Romance
Comedy
Music
Romance
Adventure
Family
Comedy
Western
Comedy
Music
Comedy
Comedy
Drama
Music
Drama
Action
Music
Romance
Comedy
Music
Romance
Family
Adventure
Comedy
Documentary
Drama
Music
Romance
Animation
Family
Adventure
Family
Fantasy
Romance
Drama
Comedy
Crime
Action
Comedy
Romance
Thriller
Animation
Family
Drama
Comedy
Romance
Family
Comedy
Adventure
Action
Comedy
Crime
Comedy
Drama
Comedy
Drama
Horror
Comedy
Adventure
Action
Thriller
Adventure
Action
Comedy
Western
Documentary
Music
Drama
Comedy
Horror
Mystery
Action
Drama
History
Adventure
Comedy
Drama
Fantasy
Horror
Science Fiction
Romance
Comedy
Comedy
Crime
Drama
Adventure
Action
Thriller
Drama
Thriller
Adventure
Action
Thriller
Romance
Drama
Drama
Action
Adventure
Thriller
Science Fiction
Adventure
Action
Thriller
Comedy
Drama
Drama
Roma

Thriller
Family
Comedy
Fantasy
Action
Adventure
Drama
Music
Drama
Comedy
Horror
Mystery
Thriller
Comedy
Romance
Drama
Thriller
Mystery
Drama
Comedy
Drama
Drama
Action
Comedy
Thriller
Crime
Romance
Drama
Crime
Drama
Romance
Comedy
Crime
Drama
Mystery
Thriller
Documentary
Drama
Thriller
Comedy
Romance
Drama
Action
Thriller
Comedy
Action
Comedy
Thriller
Drama
Romance
Action
Crime
Drama
Thriller
Action
Comedy
Science Fiction
Music
Documentary
Documentary
Drama
Action
Science Fiction
Thriller
Comedy
Drama
Romance
Documentary
History
Drama
Romance
Action
Drama
Western
Horror
Mystery
Drama
Family
Drama
Comedy
Romance
Comedy
Fantasy
Horror
Music
Thriller
Action
Drama
Western
Western
Horror
Comedy
Drama
Romance
Thriller
Horror
Mystery
Adventure
Comedy
Romance
Comedy
Drama
Romance
Comedy
Music
Comedy
Romance
Action
Adventure
Comedy
Crime
Action
Adventure
Drama
Drama
Romance
Action
Comedy
Romance
Crime
Drama
Romance
Thriller
Comedy
Drama
Drama
Romance
Drama
War
Comedy
Romance
Thriller
Drama
Myste

...

Comedy
Drama
Action
Comedy
Foreign
Drama
Drama
Comedy
Romance
Drama
Comedy
Drama
Action
Action
Documentary
Music
Comedy
Drama
Drama
Mystery
Thriller
Science Fiction
Thriller
Drama
Adventure
Action
Thriller
Family
Drama
Comedy
Comedy
Drama
Drama
Animation
Drama
Action
Thriller
Foreign
Fantasy
Animation
Science Fiction
Foreign
Western
Drama
Thriller
Thriller
Action
Animation
Science Fiction
Adventure
Crime
Drama
Adventure
Animation
Comedy
Fantasy
Drama
Crime
Action
Adventure
Animation
Action
Animation
Animation
Action
Science Fiction
Action
Animation
Fantasy
Science Fiction
Adventure
Animation
Comedy
Fantasy
Animation
Horror
Adventure
Animation
Comedy
Fantasy
Animation
Adventure
Comedy
Fantasy
Fantasy
Animation
Comedy
Adventure
Animation
Drama
Fantasy
Fantasy
Animation
Animation
Comedy
Fantasy
Animation
Science Fiction
Foreign
Action
Adventure
Animation
Science Fiction
Science Fiction
Animation
Action
War
Animation
Science Fiction
Animation
Action
Animation
Drama
Action
Comedy
Action
Ani

Thriller
Horror
Comedy
Action
Drama
Horror
Drama
Drama
Action
Crime
Thriller
Thriller
Romance
Drama
Comedy
Documentary
History
Comedy
Drama
Romance
Drama
Thriller
Romance
Drama
Comedy
Horror
Fantasy
Thriller
Mystery
Adventure
Action
Western
Drama
Action
Drama
War
Action
Comedy
Thriller
Action
Science Fiction
Drama
Romance
Comedy
Romance
Documentary
History
Documentary
Comedy
Drama
Action
Drama
Romance
Drama
Romance
Drama
Family
Romance
Western
Drama
Western
Drama
Action
Western
Family
Drama
Romance
Family
Drama
Western
Adventure
Animation
Family
Science Fiction
Action
Comedy
Action
Science Fiction
Documentary
Comedy
Comedy
Comedy
Drama
Romance
Animation
Adventure
Fantasy
Crime
Drama
Mystery
Thriller
Romance
Drama
Horror
Comedy
Drama
Comedy
Documentary
Action
Science Fiction
Thriller
Science Fiction
Action
Thriller
Animation
Science Fiction
Horror
Thriller
War
Drama
Romance
Drama
Animation
Horror
Thriller
Horror
Documentary
Documentary
Drama
Documentary
Drama
Horror
Mystery
Animation
Wa

Documentary
Science Fiction
Comedy
Drama
Adventure
Drama
Romance
Science Fiction
Documentary
Foreign
Drama
Drama
Drama
Comedy
Comedy
Horror
Mystery
Thriller
Thriller
Horror
Documentary
Horror
Action
Comedy
Documentary
Thriller
Comedy
Romance
Comedy
Romance
Foreign
Drama
Comedy
Thriller
Horror
Romance
Comedy
Documentary
Documentary
Drama
Documentary
Romance
Comedy
Drama
Drama
History
Drama
War
Comedy
Drama
Drama
Action
Foreign
Animation
Comedy
Adventure
Drama
Music
Comedy
Documentary
Documentary
Comedy
Crime
Drama
Thriller
Mystery
Comedy
Drama
Crime
Drama
Thriller
Romance
Drama
Thriller
Drama
Animation
Adventure
Animation
Comedy
Fantasy
Animation
Family
Comedy
Adventure
Drama
Animation
Comedy
Drama
Thriller
Comedy
Horror
Thriller
Horror
Horror
Drama
Action
Drama
History
Drama
Comedy
Drama
Crime
Drama
Crime
Thriller
Documentary
Crime
Drama
Mystery
Documentary
Animation
Comedy
Comedy
History
Documentary
Romance
Drama
Drama
Music
Documentary
Documentary
Horror
Drama
Comedy
Music
Crime
Dram

Adventure
Animation
Comedy
Family
Science Fiction
Drama
Romance
Comedy
Family
Drama
Adventure
Family
Thriller
Drama
Crime
Comedy
Adventure
Mystery
Drama
Science Fiction
Drama
Comedy
Action
Fantasy
Adventure
Drama
History
Drama
Action
Comedy
Drama
Thriller
Action
Comedy
Romance
Science Fiction
Thriller
Drama
Music
Animation
Thriller
Action
Adventure
Comedy
Drama
Music
Adventure
Action
Horror
Comedy
Family
Fantasy
Romance
Comedy
Documentary
Animation
Drama
Fantasy
Animation
Adventure
Comedy
Drama
Animation
Drama
Family
Drama
Romance
Drama
Comedy
Documentary
Documentary
Mystery
Thriller
Animation
Action
Crime
Thriller
Comedy
Drama
Drama
Action
Comedy
Documentary
Drama
Mystery
Comedy
Drama
Drama
Action
History
Science Fiction
Drama
Comedy
Action
Drama
Action
Adventure
Crime
Drama
Thriller
Animation
Drama
Romance
Documentary
Documentary
Drama
Horror
Science Fiction
Thriller
Romance
Crime
Drama
Mystery
Thriller
Horror
Drama
Drama
Horror
Thriller
Comedy
Comedy
Fantasy
Romance
Thriller
Horror


Comedy
Adventure
Western
Action
Drama
Comedy
Action
Thriller
Thriller
Documentary
Thriller
Horror
Documentary
Comedy
Horror
Animation
Horror
Science Fiction
Horror
Fantasy
Animation
Horror
Horror
Animation
Comedy
Science Fiction
Comedy
Romance
Fantasy
Drama
Crime
Drama
Drama
Science Fiction
Romance
Thriller
Horror
Western
Comedy
Drama
Family
Thriller
Crime
Comedy
Adventure
Animation
Horror
Horror
Mystery
Thriller
Family
Animation
Comedy
Drama
Horror
Documentary
Documentary
Fantasy
Animation
Science Fiction
Family
Animation
Adventure
Fantasy
Animation
Action
Family
Animation
Family
Comedy
Family
Comedy
Drama
Family
Animation
Family
Adventure
Fantasy
Animation
Science Fiction
Family
Adventure
Animation
Family
Fantasy
Animation
Family
Science Fiction
Animation
Comedy
Documentary
Romance
Comedy
Foreign
Drama
Foreign
Documentary
Comedy
Comedy
Comedy
Drama
Thriller
Action
Science Fiction
Thriller
Adventure
Drama
Comedy
Documentary
Fantasy
Mystery
Thriller
Comedy
Comedy
Drama
Comedy
Comedy
Co

In [27]:
df_info_pelis["generos"].apply(lambda x:len(x)).value_counts()

1    14814
2    14484
3     9493
4     3260
0     2449
5      789
6      151
7       23
8        3
Name: generos, dtype: int64

In [49]:
df["movieId"].value_counts()

356       341
296       324
318       311
593       304
260       291
         ... 
98604       1
103659      1
104419      1
115927      1
6425        1
Name: movieId, Length: 9066, dtype: int64

In [50]:
df["userId"].value_counts()

547    2391
564    1868
624    1735
15     1700
73     1610
       ... 
296      20
289      20
249      20
221      20
1        20
Name: userId, Length: 671, dtype: int64

In [10]:
df_entrenar=df.pivot_table(values="rating",index="userId",columns="movieId")


In [11]:
df_entrenar.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [13]:
df_entrenar2=df_entrenar.fillna(0)

In [14]:
df_entrenar2.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
distancias=cosine_similarity(df_entrenar2)

In [16]:
distancias[0]

array([1.        , 0.        , 0.        , 0.07448245, 0.01681799,
       0.        , 0.08388416, 0.        , 0.01284289, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.05938727,
       0.        , 0.02107287, 0.        , 0.08563118, 0.01656352,
       0.05733056, 0.0436822 , 0.07024269, 0.        , 0.01491541,
       0.00262104, 0.03501505, 0.        , 0.        , 0.06130103,
       0.02379292, 0.03170691, 0.00821267, 0.08459736, 0.13058496,
       0.01652849, 0.        , 0.03282542, 0.0203909 , 0.        ,
       0.06833225, 0.        , 0.04105418, 0.        , 0.        ,
       0.        , 0.        , 0.02081219, 0.07333877, 0.        ,
       0.        , 0.        , 0.        , 0.03782836, 0.        ,
       0.03844173, 0.06430548, 0.        , 0.01381982, 0.        ,
       0.01399113, 0.        , 0.        , 0.        , 0.        ,
       0.04528371, 0.        , 0.00203034, 0.        , 0.03191959,
       0.        , 0.        , 0.08815485, 0.        , 0.04607

In [17]:
distancias=np.where(distancias==1,-1,distancias)

In [18]:
np.argpartition(distancias[0],-4)[-5:]

array([484, 309, 633, 324, 340], dtype=int64)

In [19]:
def devuelve_similares(indice,total_similares=5,distancias=distancias):
    return np.argpartition(distancias[indice],-total_similares)[-total_similares:]

Metamos, por ejemplo, al consumidor número 50:

In [20]:
similares_50=devuelve_similares(indice=50)

In [21]:
similares_50

array([655, 475, 625, 668, 188], dtype=int64)

Veamos ahora las películas que le gustan a nuestro 50:

In [22]:
df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [23]:
df_puntuaciones_50=df[df["userId"]==50][["movieId","rating"]]
df_puntuaciones_50

,movieId,rating
7987,10,4.0
7988,21,3.0
7989,39,2.0
7990,47,3.0
7991,95,3.0
7992,110,4.0
7993,150,3.0
7994,160,3.0
7995,161,4.0
7996,165,4.0


In [24]:
df_info_pelis[df_info_pelis["id"].isin(df[df["userId"]==50]["movieId"].values)]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [25]:
df_info_pelis["id"].dtype

dtype('O')

In [26]:
df["movieId"].dtype

dtype('int64')

In [27]:
df_info_pelis["id"].unique()

array(['862', '8844', '15602', ..., '67758', '227506', '461257'],
      dtype=object)

In [28]:
df["movieId"]=df["movieId"].astype("str")

In [29]:
df_pelis_50=df_info_pelis[df_info_pelis["id"].isin(df[df["userId"]==50]["movieId"].values)]
df_pelis_50

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
286,False,"{'id': 300546, 'name': 'Once were Warriors Col...",0,"[{'id': 18, 'name': 'Drama'}]",NaN,527,tt0110729,en,Once Were Warriors,A drama about a Maori family lving in Auckland...,...,1994-09-02,2.201126e+06,99.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"A family in crisis, a life in chaos... Nothing...",Once Were Warriors,False,7.6,106.0
302,False,"{'id': 131, 'name': 'Three Colors Collection',...",0,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",NaN,110,tt0111495,fr,Trois couleurs : Rouge,Red This is the third film from the trilogy by...,...,1994-05-27,0.000000e+00,99.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Three Colors: Red,False,7.8,246.0
385,False,NaN,45000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,315,tt0059170,en,"Faster, Pussycat! Kill! Kill!",Three strippers seeking thrills encounter a yo...,...,1965-08-06,0.000000e+00,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Go-Go For a Wild Ride With the ACTION GIRLS!,"Faster, Pussycat! Kill! Kill!",False,6.5,59.0
1029,False,NaN,14500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.romeoandjuliet.com/,454,tt0117509,en,Romeo + Juliet,In director Baz Luhrmann's contemporary take o...,...,1996-10-31,1.472988e+08,120.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,My only love sprung from my only hate.,Romeo + Juliet,False,6.7,1406.0
1163,False,NaN,2200000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",NaN,185,tt0066921,en,A Clockwork Orange,Demonic gang-leader Alex goes on the spree of ...,...,1971-12-18,2.658900e+07,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Being the adventures of a young man whose prin...,A Clockwork Orange,False,8.0,3432.0
1176,False,"{'id': 119674, 'name': 'Psycho Collection', 'p...",806948,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",NaN,539,tt0054215,en,Psycho,When larcenous real estate clerk Marion Crane ...,...,1960-06-16,3.200000e+07,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The master of suspense moves his cameras into ...,Psycho,False,8.3,2405.0
1234,False,NaN,3500000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,339,tt0102536,en,Night on Earth,An anthology of 5 different cab drivers in 5 A...,...,1991-10-03,2.015810e+06,129.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Five Taxis. Five Cities. One Night.,Night on Earth,False,7.5,165.0
1299,False,"{'id': 8581, 'name': 'A Nightmare on Elm Stree...",1800000,"[{'id': 27, 'name': 'Horror'}]",NaN,377,tt0087800,en,A Nightmare on Elm Street,Teenagers in a small town are dropping like fl...,...,1984-11-14,2.550451e+07,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If Nancy Doesn't Wake Up Screaming, She Won't ...",A Nightmare on Elm Street,False,7.2,1212.0
1639,False,NaN,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.titanicmovie.com,597,tt0120338,en,Titanic,"84 years later, a 101-year-old woman named Ros...",...,1997-11-18,1.845034e+09,194.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.5,7770.0
1808,False,NaN,140000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,95,tt0120591,en,Armageddon,When an asteroid threatens to collide with Ear...,...,1998-07-01,5.537996e+08,151.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Earth's Darkest Day Will Be Man's Finest Hour,Armageddon,False,6.5,2540.0


LLevamos las puntuaciones al df anterior:

In [30]:
df_pelis_50=df_pelis_50.merge(df_puntuaciones_50,left_on="id", right_on="movieId")[["title","id","rating"]]
df_pelis_50

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

Metemos todos los pasos anteriores en funciones:

In [32]:

##Función que obtiene las películas para un índice concreto:
def filtro_por_Id(indice,df_info_pelis=df_info_pelis,df=df):
    return df_info_pelis[df_info_pelis["id"].isin(df[df["userId"]==indice]["movieId"].values)][["title","id"]]

##Función para añadir al filtrado las puntuaciones. El valor de limitando indica
## si hay que limitar puntuaciones. Esta función internamnete llamará a la de arriba:

def anade_puntuaciones(indice,df=df,limitando = 0):
    df_salida=df[df["userId"]==50][["movieId","rating"]]
    df_salida=filtro_por_Id(indice=indice).merge(df_salida,left_on="id", right_on="movieId")[["title","id","rating"]]
    df_salida=df_salida[df_salida["rating"]>=limitando]
    df_salida["indice"]=indice
    return df_salida

##Replicamos aquí la función que obtiene los más cercanos:

def devuelve_similares1(indice,total_similares=5,distancias=distancias):
    return np.argpartition(distancias[indice],-total_similares)[-total_similares:]

###Aquí aplicamos ahora el proceso completo que irá llamando a las funciones de arriba:

def saca_recomendaciones(indice,df=df,total_similares =5,max_rating=4,filtro_solo_rec=False):
    ###Paso 1: Sacamos los argumentos asociados
    indices_aux=devuelve_similares1(indice,total_similares)
    ###Paso 2_ Sacamos las puntuaciens y películas del índicie
    pelis_salida=anade_puntuaciones(indice,limitando=0)
    ###Paso 3_ Añadimos las películas asociadas a los índices:
    lista_dfs=[anade_puntuaciones(x, limitando=max_rating) for x in indices_aux]
    for comentarista in lista_dfs:
        try:
            saca_indice=comentarista["indice"].values[0]
            pelis_salida=pelis_salida.merge(comentarista, on="id",suffixes=(None,saca_indice),how="outer")
        except:
            pass
    ##El siguiente filtro se quedará solo con las recomendaciones:
    if filtro_solo_rec:
        pelis_salida=pelis_salida[pelis_salida["title"].isnull()]
    #pelis_salida=pd.concat([pelis_salida]+lista_dfs,axis=1)
    return pelis_salida


In [33]:
saca_recomendaciones(475,filtro_solo_rec=False)

,title,id,rating,indice,title188,rating188,indice188,title189,rating189,indice189,title625,rating625,indice625,title385,rating385,indice385
0,Three Colors: Red,110,4.0,475.0,Three Colors: Red,4.0,188.0,NaN,NaN,NaN,NaN,NaN,NaN,Three Colors: Red,4.0,385.0
1,Monsoon Wedding,480,4.0,475.0,Monsoon Wedding,4.0,188.0,NaN,NaN,NaN,NaN,NaN,NaN,Monsoon Wedding,4.0,385.0
2,Terminator 3: Rise of the Machines,296,4.0,475.0,Terminator 3: Rise of the Machines,4.0,188.0,NaN,NaN,NaN,NaN,NaN,NaN,Terminator 3: Rise of the Machines,4.0,385.0
3,Grill Point,316,3.0,475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,527,NaN,NaN,Once Were Warriors,4.0,188.0,NaN,NaN,NaN,Once Were Warriors,4.0,625.0,Once Were Warriors,4.0,385.0
5,NaN,339,NaN,NaN,Night on Earth,4.0,188.0,NaN,NaN,NaN,NaN,NaN,NaN,Night on Earth,4.0,385.0
6,NaN,165,NaN,NaN,Back to the Future Part II,4.0,188.0,NaN,NaN,NaN,NaN,NaN,NaN,Back to the Future Part II,4.0,385.0
7,NaN,780,NaN,NaN,The Passion of Joan of Arc,4.0,188.0,The Passion of Joan of Arc,4.0,189.0,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,292,NaN,NaN,Dave Chappelle's Block Party,4.0,188.0,NaN,NaN,NaN,NaN,NaN,NaN,Dave Chappelle's Block Party,4.0,385.0
9,NaN,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ocean's Eleven,4.0,385.0


In [123]:
devuelve_similares1(475)

array([188, 189, 625, 385, 540], dtype=int64)

Añadimos las películas de la recomendación:

In [126]:
anade_puntuaciones(188)

,title,id,rating,indice
0,Once Were Warriors,527,4.0,188
1,Three Colors: Red,110,4.0,188
2,Romeo + Juliet,454,3.0,188
3,Psycho,539,3.0,188
4,Night on Earth,339,4.0,188
5,A Nightmare on Elm Street,377,3.0,188
6,Titanic,597,3.0,188
7,Rain Man,380,3.0,188
8,Back to the Future Part II,165,4.0,188
9,Notting Hill,509,3.0,188


In [48]:
 modelo = NMF(n_components=2, init='random', random_state=0)

In [12]:
df_entrenar2=np.where(df_entrenar2==-1,1,df_entrenar2)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [13]:
modelo.fit(df_entrenar2)

NMF(init='random', n_components=2, random_state=0)